# On average, do males or females spend more money per order? Including discount and not including discount.

In [1]:
import pandas as pd
import numpy as np

import scipy.stats as scs
from scipy import stats 
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.multicomp import (pairwise_tukeyhsd, MultiComparison)
import math
import statsmodels.api as sm
from statsmodels.formula.api import ols

import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

import sqlite3

In [2]:
conn = sqlite3.connect("./Northwind_small.sqlite")
cursor = conn.cursor()

In [3]:
query = '''SELECT c.ID AS CustomerID, c.ContactName, o.ID AS OrderID, od.UnitPrice, od.Quantity, od.Discount
                            FROM OrderDetail od
                            JOIN [Order] o ON od.OrderID = o.ID
                            JOIN Customer c ON o.CustomerID = c.ID
                            '''
df2 = pd.read_sql(query, conn)

print('\nTable info')
df2.info()
print('\nSample data')
df2.head()


Table info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2078 entries, 0 to 2077
Data columns (total 6 columns):
CustomerID     2078 non-null object
ContactName    2078 non-null object
OrderID        2078 non-null int64
UnitPrice      2078 non-null float64
Quantity       2078 non-null int64
Discount       2078 non-null float64
dtypes: float64(2), int64(2), object(2)
memory usage: 97.5+ KB

Sample data


,CustomerID,ContactName,OrderID,UnitPrice,Quantity,Discount
0,VINET,Paul Henriot,10248,14.0,12,0.0
1,VINET,Paul Henriot,10248,9.8,10,0.0
2,VINET,Paul Henriot,10248,34.8,5,0.0
3,TOMSP,Karin Josephs,10249,18.6,9,0.0
4,TOMSP,Karin Josephs,10249,42.4,40,0.0


In [4]:
df2.ContactName.unique()

array(['Paul Henriot', 'Karin Josephs', 'Mario Pontes', 'Mary Saveley',
       'Pascale Cartrain', 'Yang Wang', 'Michael Holz', 'Paula Parente',
       'Carlos Hernández', 'Roland Mendel', 'Francisco Chang',
       'Henriette Pfalzheim', 'Bernardo Batista', 'Paula Wilson',
       'Maria Larsson', 'Frédérique Citeaux', 'Pirkko Koskitalo',
       'Peter Franken', 'Manuel Pereira', 'Karl Jablonski',
       'Art Braunschweiger', 'Horst Kloss', 'Giovanni Rovelli',
       'Miguel Angel Paolino', 'Alexander Feuer', 'Christina Berglund',
       'Renate Messner', 'Alejandra Camino', 'Carlos González',
       'Janete Limeira', 'Maurizio Moroni', 'Victoria Ashworth',
       'Pedro Afonso', 'Anabela Domingues', 'Patricia McKenna',
       'Rita Müller', 'José Pedro Freyre', 'Rene Phillips', 'Fran Wilson',
       'Ana Trujillo', 'Liz Nixon', 'Helen Bennett',
       'Guillermo Fernández', 'Philip Cramer', 'Jose Pavarotti',
       'Martín Sommer', 'Lino Rodriguez', 'Laurence Lebihan',
       'Jean Fre

In [5]:
females = ['Karin Josephs','Mary Saveley','Paula Parente','Henriette Pfalzheim',
          'Paula Wilson','Maria Larsson','Christina Berglund','Renate Messner', 
           'Alejandra Camino','Janete Limeira','Victoria Ashworth','Anabela Domingues', 
           'Patricia McKenna','Rita Müller','Ana Trujillo', 'Liz Nixon', 'Helen Bennett',
          'Isabel de Castro', 'Jytte Petersen','Aria Cruz', 'Annette Roulet',
          'Ann Devon','Elizabeth Lincoln','Martine Rancé','Elizabeth Brown',
          'Hanna Moos','Catherine Dewey']
df2['Gender'] = np.where(df2['ContactName'].isin(females), 'female', 'male')
df2.head()

,CustomerID,ContactName,OrderID,UnitPrice,Quantity,Discount,Gender
0,VINET,Paul Henriot,10248,14.0,12,0.0,male
1,VINET,Paul Henriot,10248,9.8,10,0.0,male
2,VINET,Paul Henriot,10248,34.8,5,0.0,male
3,TOMSP,Karin Josephs,10249,18.6,9,0.0,female
4,TOMSP,Karin Josephs,10249,42.4,40,0.0,female


In [6]:
# Now that I grouped the customers by gender to create a new column,
# I can begin to explore which gender spends the most money per order.


In [ ]:
df2['GrossPrice'] = np.multiply(df2['Quantity'],df2['UnitPrice'])
df2['NetPrice'] = np.multiply(df2['GrossPrice'],df2['Discount'])
df2['NetPrice'] = np.subtract(df2['GrossPrice'],df2['NetPrice'])
df2.head()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8,8))
#fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8,8), colormap='cool')
df2.groupby(['Gender'])['Gender'].count().plot(kind='barh', color=['pink','blue'], ax=ax1)
df2.groupby(['Gender'])['Quantity'].mean().plot(kind='barh', color=['pink','blue'], ax=ax2)
df2.groupby(['Gender'])['NetPrice'].mean().plot(kind='barh', color=['pink','blue'], ax=ax3)
ax1.set_title('Number Of Orders By Gender')
ax1.set_ylabel('')
ax1.set_xlabel('# Of Orders')
ax2.set_title('Average Quantity Ordered By Gender')
ax2.set_ylabel('')
ax2.set_xlabel('Quantity Ordered')
ax3.set_title('Average Revenue By Gender')
ax3.set_ylabel('')
ax3.set_xlabel('Average Revenue')
fig.subplots_adjust(hspace=0.4);

In [ ]:
df2_male = df2.loc[df2['Gender'] == 'male']
df2_male.describe()

In [ ]:
df2_female = df2.loc[df2['Gender'] == 'female']
df2_female.describe()

In [ ]:
# I observed from the data that the total count of orders by males is 1402 and 676 for females.
# Since there is almost double the data on males than females,
# I will try and represent this using an average of (100 order)   

In [ ]:
#plt.figure(figsize=(6,4))
#df2_male['NetPrice'].plot(kind='hist', alpha=0.6, legend=True, label='Male')
#df2_female['NetPrice'].plot(kind='hist',  alpha=0.4, legend=True, label='Female')
#plt.title('Revenue Per Gender');

In [ ]:
#plt.figure(figsize=(8,5))
#for gender in set(df2.Gender):
#    gender_group = df2.loc[df2['Gender'] == gender]
#    sns.distplot(gender_group['NetPrice'], hist_kws=dict(alpha=0.2), label=gender)
#    plt.legend()
#    plt.xlabel('NetPrice Per Gender')

# Statistical Analysis

In [ ]:
# One sample t-test

# h0: No difference in money spent on purchases by Males & Females 
# ha: Males spent the most money on purchases

# List Assumptions:
# Data is Normal Distribution 
# Data is Randomly Sampled
# Data is Independent

In [ ]:
def one_sample_ttest(sample, popmean, alpha):

    # Use eye test on distribution to check for normality 
    sns.set(color_codes=True)
    sns.set(rc={'figure.figsize':(12,10)})
    sns.distplot(sample)
    
    # Population mean 
    mu = popmean
    
    # Sample mean (x̄) using NumPy mean()
    x_bar= sample.mean()

    # std dev of our sample
    sigma = np.std(sample)
    
    # DF
    df = len(sample) - 1
    
    #Find critical t-value
    t_crit = stats.t.ppf(1 - alpha, df=df)
    
    #Find p-value and t-value
    results = stats.ttest_1samp(a= sample, popmean= mu)         
    
    if (results[0]>t_crit) and (results[1]<alpha):
        print ("We reject the null hypothesis. Results are statistically significant with t-value =", 
                round(results[0], 2), "critical t-value =", t_crit, "and p-value =", np.round((results[1]), 6))
    else:
        print ("We fail to reject the null hypothesis with a t-value =", 
                round(results[0], 2), ", critical t-value =", t_crit, "and p-value =", np.round((results[1]), 6))

In [ ]:
sample2 = df2_male.NetPrice
popmean2 = df2_female.NetPrice.mean()
alpha = 0.05
one_sample_ttest(sample2, popmean2, alpha)

In [ ]:
def Cohen_d(group1, group2):

    diff = group1.mean() - group2.mean()

    n1, n2 = len(group1), len(group2)
    var1 = group1.var()
    var2 = group2.var()

    # Calculate the pooled variance threshold 
    pooled_var = (n1 * var1 + n2 * var2) / (n1 + n2)
    
    # Calculate Cohen's D
    d = diff / np.sqrt(pooled_var)
    
    return d

In [ ]:
males = df2_male.NetPrice
females = df2_female.NetPrice
Cohen_d(males, females)
# effect size is small with a value of ~.10

In [ ]:
# As stated above, males make more orders than women. Also, Males spend more money on their orders than women! Not the result I would have expected!
# We reject the null hypothesis and conclude that males purchase more items and spend more money per order than females.
# By rejecting the null hypothesis, we are saying that there is a difference in money spent on purchases by gender.

In [ ]:
# test the sample means
#stats.ttest_ind(sample1, sample2)
#
#plt.figure("Test Samples")
#sns.distplot(sample1, label='Sample1') 
#sns.distplot(sample2, label='Sample2')
#plt.legend()
#plt.show()


In [ ]:
#https://github.com/AllaGonzalez/Hypothesis_Testing_Using_Northwind_Database/blob/master/Project%20Notebook.ipynb